In [1]:
import json

# Load the JSON file as a dictionary
file_path = "products_bought_expanded_with_rating.json"

with open(file_path, "r") as file:
    data = json.load(file)

In [2]:
# from pyspark.sql import SparkSession
# from tqdm import tqdm

# # Create a Spark session
# spark = SparkSession.builder.appName("SaveDictionary").getOrCreate()

# # Convert dictionary to a list of tuples
# data_list = []

# # Wrap `data.items()` with `tqdm` for a progress bar
# for key, value in tqdm(data.items(), desc="Processing Items", total=len(data)):
#     data_list.append((key, value))
    
# del data

# # Create a DataFrame
# df = spark.createDataFrame(data_list, schema=["user_id", "products"])

# # Save the DataFrame as Parquet
# output_path = "user_products_parquet"
# df.write.parquet(output_path)

# print(f"Dictionary saved as Parquet at {output_path}")

In [4]:
# Display the first 10 key-value pairs
data_1000 = list(data.items())[:1000]

In [ ]:
from pyspark.sql import SparkSession
from itertools import combinations

spark = SparkSession.builder \
    .appName("LargeDataProcessing") \
    .config("spark.executor.memory", "20g") \
    .config("spark.executor.instances", "5") \
    .config("spark.driver.memory", "20g") \
    .getOrCreate()

# Path to JSON file
json_file_path = "products_bought_expanded.json"

# Minimum support threshold (e.g., 0.5 for 50% of transactions)
min_support = 0.5

# Step 1: Load the JSON file
# Assuming JSON structure: [{"user_id": "u1", "products": [{"parent_asin": "P1"}, {"parent_asin": "P2"}]}, ...]
# data = spark.read.json(json_file_path)

# Step 1: Transform data for Spark
transactions = [
    (user, [item["parent_asin"] for item in items]) for user, items in data.items()
]

# Convert to RDD
transactions_rdd = spark.sparkContext.parallelize(transactions, numSlices=10000)

# Step 2: Compute total number of transactions
total_transactions = transactions_rdd.count()

# Minimum support threshold (e.g., 50%)
min_support = 0.3

# Step 3: Generate frequent itemsets
# Single item frequencies
single_item_counts = transactions_rdd.flatMap(lambda x: [(item, 1) for item in x[1]]) \
    .reduceByKey(lambda a, b: a + b)

# Filter single items by minimum support
frequent_single_items = single_item_counts.filter(lambda x: x[1] / total_transactions >= min_support)

# Collect frequent single items
frequent_items = frequent_single_items.map(lambda x: x[0]).collect()

# Step 4: Generate and count item combinations iteratively
frequent_itemsets = []
current_itemsets = frequent_items

k = 2
while current_itemsets:
    # Generate candidate itemsets of size k
    candidate_itemsets = transactions_rdd.flatMap(
        lambda x: [tuple(sorted(combo)) for combo in combinations(x[1], k) if all(item in frequent_items for item in combo)]
    ).map(lambda x: (x, 1))
    
    # Count occurrences of each candidate itemset
    candidate_counts = candidate_itemsets.reduceByKey(lambda a, b: a + b)
    
    # Filter by minimum support
    current_frequent_itemsets = candidate_counts.filter(lambda x: x[1] / total_transactions >= min_support)
    
    # Collect frequent itemsets of size k
    current_itemsets = current_frequent_itemsets.map(lambda x: x[0]).collect()
    
    # Add to the final list of frequent itemsets
    frequent_itemsets.extend(current_itemsets)
    
    k += 1

# Output results
print("Frequent Itemsets:")
for itemset in frequent_itemsets:
    print(itemset)


24/11/19 01:46:01 WARN TaskSetManager: Stage 15 contains a task of very large size (3528 KiB). The maximum recommended task size is 1000 KiB.
24/11/19 01:46:48 WARN TaskSetManager: Stage 16 contains a task of very large size (3528 KiB). The maximum recommended task size is 1000 KiB.


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_list, col
from itertools import combinations

# Step 1: Create a Spark session
spark = SparkSession.builder.appName("FrequentlyBoughtTogether").getOrCreate()

# Path to JSON file
json_file_path = "products_bought_expanded.json"

# Minimum support threshold (e.g., 0.5 for 50% of transactions)
min_support = 0.5

# Step 1: Load the JSON file
# Assuming JSON structure: [{"user_id": "u1", "products": [{"parent_asin": "P1"}, {"parent_asin": "P2"}]}, ...]
data = spark.read.json(json_file_path)

print(data)

# Step 2: Convert dictionary to a list of tuples for DataFrame creation
data_list = [(user_id, product["parent_asin"]) for user_id, products in data.items() for product in products]

# Create a DataFrame
df = spark.createDataFrame(data_list, ["user_id", "parent_asin"])

# Step 3: Group by user_id and collect parent_asin into a list
user_products_df = df.groupBy("user_id").agg(
    collect_list("parent_asin").alias("products")
)

# Step 4: Generate product pairs for each user
def generate_pairs(products):
    return list(combinations(sorted(set(products)), 2))  # Remove duplicates and sort

pair_rdd = user_products_df.rdd.flatMap(
    lambda row: [(pair, 1) for pair in generate_pairs(row["products"])]
)

# Step 5: Convert RDD to DataFrame and count occurrences of each pair
pair_df = pair_rdd.toDF(["product_pair", "count"]).groupBy("product_pair").sum("count")

# Step 6: Filter pairs by a threshold (e.g., 2)
frequent_pairs_df = pair_df.filter(col("sum(count)") >= 2)

# Show frequently bought together pairs
frequent_pairs_df.show()

DataFrame[_corrupt_record: string]


AttributeError: 'DataFrame' object has no attribute 'items'

In [11]:
from pyspark.sql import SparkSession
import json

# Create a Spark session
spark = SparkSession.builder.appName("LoadJSON").getOrCreate()

# Load JSON file into an RDD
rdd = spark.sparkContext.textFile("products_bought_expanded.json")

# Parse JSON strings into Python dictionaries
rdd_dict = rdd.map(lambda x: json.loads(x))  # Import json library

# Collect or process further
print(rdd_dict.collect())

24/11/18 17:17:05 ERROR Executor: Exception in task 64.0 in stage 14.0 (TID 948)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/uufs/chpc.utah.edu/common/home/u1472614/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/uufs/chpc.utah.edu/common/home/u1472614/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/uufs/chpc.utah.edu/common/home/u1472614/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/uufs/chpc.utah.edu/common/home/u1472614/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/scratch/local/u1472614/2303750/ipykernel_3624648/414682744

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 64 in stage 14.0 failed 1 times, most recent failure: Lost task 64.0 in stage 14.0 (TID 948) (notch160.ipoib.int.chpc.utah.edu executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/uufs/chpc.utah.edu/common/home/u1472614/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/uufs/chpc.utah.edu/common/home/u1472614/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/uufs/chpc.utah.edu/common/home/u1472614/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/